In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

/tmp/ipykernel_20760/956813560.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
#chack for updated files\n,
%load_ext autoreload
%autoreload 2
#import packages
from __future__ import division

%load_ext line_profiler
import numpy as np
import glob, json
import pickle
import os as os_pack
import matplotlib.pyplot as plt
import corner
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import healpy as hp
import os, glob, json, pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as sl
import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals
import enterprise.constants as const
from enterprise_extensions import blocks
from enterprise_extensions import models as ee_models
from enterprise_extensions import model_utils as ee_model_utils
from enterprise_extensions import model_orfs
from enterprise_extensions.hypermodel import HyperModel
from enterprise_extensions.frequentist import optimal_statistic as opt_stat
from enterprise_extensions import sampler as ee_sampler
from enterprise.signals.signal_base import LogLikelihood
#import enterprise_wavelets as models
from enterprise.signals.deterministic_signals import Deterministic
from enterprise.signals.parameter import function
from la_forge.core import Core
from la_forge.diagnostics import plot_chains
from la_forge import rednoise
import la_forge
import corner
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
import re

In [4]:
def figsize(scale):
    fig_width_pt = 513.17 #469.755                  # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*golden_mean              # height in inches
    fig_size = [2 * fig_width,2 * fig_height]       #1* scaling factor for 2 collem paper
    return fig_size
plt.rcParams.update(plt.rcParamsDefault)

params = {'backend': 'pdf',
        'axes.labelsize': 18,
        'lines.markersize': 4,
        'font.size': 12,
        'xtick.major.size':6,
        'xtick.minor.size':3,  
        'ytick.major.size':6,
        'ytick.minor.size':3, 
        'xtick.major.width':0.5,
        'ytick.major.width':0.5,
        'xtick.minor.width':0.5,
        'ytick.minor.width':0.5,
        'lines.markeredgewidth':1,
        'axes.linewidth':1.2,
        'legend.fontsize': 13,
        'xtick.labelsize': 14,
        'ytick.labelsize': 14,
        'savefig.dpi':200,
        'path.simplify':True,
        'font.family': 'serif',
        #'font.serif':'Times',
        #'text.latex.preamble': [r'\usepackage{amsmath}'],
        #'text.usetex':True,
        'figure.figsize': figsize(0.5)}
#plt.style.use('dark_background')
plt.style.use('default')
plt.rcParams.update(params)

In [4]:
#Dataset w/out RN w/ parabolic encounter @ 30 Mpc
# with open("/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/30Mpc_parabolic_flyby/noRN/Psrs_pkl_10yrs.pkl", 'rb') as f:
#     psrs_sim = pickle.load(f)

#Dataset w/ RN w/ parabolic encounter @ 60 Mpc
# with open("/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/60Mpc_parabolic_flyby/RN/Psrs_pkl_10yrs.pkl", 'rb') as f:
#     psrs_sim = pickle.load(f)

with open("/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/Simple_20Psr_30Mpc_parabolic_flyby/noCURN/Psrs_pkl.pkl", 'rb') as f:
    psrs_sim = pickle.load(f)

In [5]:
len(psrs_sim)

20

In [6]:
#Loading in pickle and noise files
#15 year A4Cast noise file
# noise_file_sim = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/quickburst_test_15yrAstro4Cast_parabolic_3Mpc_60M/v1p1_all_dict.json"

#Simple 20PSR CURN dataset noise file
noise_file_sim = "/home/reyna/BayesHopperBurst/BenceData/with_burst/params_simulated.json"
with open(noise_file_sim, 'rb') as h:
    noise_params = json.load(h)
noise_params

{'JPSR00_efac': 1.0,
 'JPSR01_efac': 1.0,
 'JPSR02_efac': 1.0,
 'JPSR03_efac': 1.0,
 'JPSR04_efac': 1.0,
 'JPSR05_efac': 1.0,
 'JPSR06_efac': 1.0,
 'JPSR07_efac': 1.0,
 'JPSR08_efac': 1.0,
 'JPSR09_efac': 1.0,
 'JPSR10_efac': 1.0,
 'JPSR11_efac': 1.0,
 'JPSR12_efac': 1.0,
 'JPSR13_efac': 1.0,
 'JPSR14_efac': 1.0,
 'JPSR15_efac': 1.0,
 'JPSR16_efac': 1.0,
 'JPSR17_efac': 1.0,
 'JPSR18_efac': 1.0,
 'JPSR19_efac': 1.0,
 'JPSR00_log10_equad': -17,
 'JPSR01_log10_equad': -17,
 'JPSR02_log10_equad': -17,
 'JPSR03_log10_equad': -17,
 'JPSR04_log10_equad': -17,
 'JPSR05_log10_equad': -17,
 'JPSR06_log10_equad': -17,
 'JPSR07_log10_equad': -17,
 'JPSR08_log10_equad': -17,
 'JPSR09_log10_equad': -17,
 'JPSR10_log10_equad': -17,
 'JPSR11_log10_equad': -17,
 'JPSR12_log10_equad': -17,
 'JPSR13_log10_equad': -17,
 'JPSR14_log10_equad': -17,
 'JPSR15_log10_equad': -17,
 'JPSR16_log10_equad': -17,
 'JPSR17_log10_equad': -17,
 'JPSR18_log10_equad': -17,
 'JPSR19_log10_equad': -17}

In [7]:
#remove nmodel from noise_params
#del noise_params['nmodel']
#Injected CURN parameter values for 15 year 3Mpc 60Msolar parabolic flyby 
Amp = -14.22 #-14.6 #-16.27
gamma = 13/3 #6.6

noise_params['gw_crn_gamma'] = gamma
noise_params['gw_crn_log10_A'] = Amp

noise_dict_sim = {} #new noise dict to conver the simulated noise file into a readable formate
for k,v in noise_params.items():
    if '_equad' in k:
        noise_dict_sim.update({k.split('_equad')[0] + '_t2equad': v})
    else:
        noise_dict_sim.update({k : v})
noise_params = noise_dict_sim

In [8]:
noise_params

{'JPSR00_efac': 1.0,
 'JPSR01_efac': 1.0,
 'JPSR02_efac': 1.0,
 'JPSR03_efac': 1.0,
 'JPSR04_efac': 1.0,
 'JPSR05_efac': 1.0,
 'JPSR06_efac': 1.0,
 'JPSR07_efac': 1.0,
 'JPSR08_efac': 1.0,
 'JPSR09_efac': 1.0,
 'JPSR10_efac': 1.0,
 'JPSR11_efac': 1.0,
 'JPSR12_efac': 1.0,
 'JPSR13_efac': 1.0,
 'JPSR14_efac': 1.0,
 'JPSR15_efac': 1.0,
 'JPSR16_efac': 1.0,
 'JPSR17_efac': 1.0,
 'JPSR18_efac': 1.0,
 'JPSR19_efac': 1.0,
 'JPSR00_log10_t2equad': -17,
 'JPSR01_log10_t2equad': -17,
 'JPSR02_log10_t2equad': -17,
 'JPSR03_log10_t2equad': -17,
 'JPSR04_log10_t2equad': -17,
 'JPSR05_log10_t2equad': -17,
 'JPSR06_log10_t2equad': -17,
 'JPSR07_log10_t2equad': -17,
 'JPSR08_log10_t2equad': -17,
 'JPSR09_log10_t2equad': -17,
 'JPSR10_log10_t2equad': -17,
 'JPSR11_log10_t2equad': -17,
 'JPSR12_log10_t2equad': -17,
 'JPSR13_log10_t2equad': -17,
 'JPSR14_log10_t2equad': -17,
 'JPSR15_log10_t2equad': -17,
 'JPSR16_log10_t2equad': -17,
 'JPSR17_log10_t2equad': -17,
 'JPSR18_log10_t2equad': -17,
 'JPSR19_l

In [9]:
#Setting dataset max time and reference time
maximum = 0
minimum = np.inf
for psr in psrs_sim:
    if psr.toas.max() > maximum:
        maximum = psr.toas.max()
    if psr.toas.min() < minimum:
        minimum = psr.toas.min()


#Sets reference time
tref = minimum
print(tref/3600/24/365)
t0_max = (maximum - minimum)/365/24/3600
print(t0_max)

145.2054684686619
9.945234130667286


In [1]:
os.chdir('/home/reyna/BayesHopperBurst/QuickBurst/')
import QuickBurst_MCMC_testing as QuickBurst_MCMC
N_slow=int(1e5)
n_fish_update = int(N_slow/2)

T_max = 4 #2
n_chain = 5 #3

#Prior bounds on shape params
tau_min = 0.2
tau_max = 3.0 #5.0
f_max = 1e-7
f_min = 1e-8 #3.5e-9

#Simple 30Mpc dataset
ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/Simple_20Psr_30Mpc_parabolic_flyby/CURN_13_3/simple_wavelet_tau_scan_tau_max_3yr.pkl"
glitch_ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/Simple_20Psr_30Mpc_parabolic_flyby/CURN_13_3/simple_glitch_tau_scan_tau_max_3yr.pkl"


#30 Mpc dataset
# ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/30Mpc_parabolic_flyby/noRN/wavelet_tauscan_15yr-Astro4Cast_parabolic_30Mpc_NoRN.pkl"
# glitch_ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/30Mpc_parabolic_flyby/noRN/glitch_tauscan_15yr-Astro4Cast_parabolic_30Mpc_NoRN.pkl"

#60Mpc dataaset
# ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/60Mpc_parabolic_flyby/RN/10yrs_psrs_tauscans/wavelet_tauscan_fixed_NaNs.pkl"
# glitch_ts_file = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Data/Paper_Datasets/60Mpc_parabolic_flyby/RN/10yrs_psrs_tauscans/glitch_tauscan_fixed_NaNs.pkl"

#noisedict_file = "/home/reyna/BayesHopperBurst/BenceData/with_burst/params_simulated.json"
#RN_start_file = "../12p5yr-like_data/RN_start_values.npz"

#start_file = "/home/reyna/BayesHopperBurst/QuickBurst/Testing/2_wavelet_sim/chain_QB2.npz"
filepath = "/home/reyna/BayesHopperBurst/QuickBurst/15_year_A4Cast/Chains/QuickBurst/Paper_chains/Simple_20PSR_30Mpc_parabolic_flyby/CURN_13_3/Enterprise_comparison/"
os.makedirs(filepath, exist_ok = True)
savepath = filepath + "5w3g_4maxTemp_Tau3yr_Alljumps" #only saving a chain

attributes = ['resres_logdet', 'glitch_prm', 'wavelet_prm', 'glitch_pulsars', 'params', 'NN', 'sigmas'] #'Nvecs_previous', 'Nvecs', 'MMs',
#step_array, likelihood_attributes
samples, acc_fraction, swap_record, rj_record, ptas, ent_ptas, log_likelihood, ent_lnlikelihood, betas, PT_acc = QuickBurst_MCMC.run_bhb(N_slow, T_max, n_chain, psrs_sim,
                                                                    max_n_wavelet=5,
                                                                    min_n_wavelet=0,
                                                                    n_wavelet_start=2,
                                                                    RJ_weight=2,
                                                                    glitch_RJ_weight=2,
                                                                    regular_weight=2,
                                                                    noise_jump_weight=2,
                                                                    PT_swap_weight=2,
                                                                    tau_scan_proposal_weight=2,
                                                                    glitch_tau_scan_proposal_weight=2,
                                                                    tau_scan_file=ts_file,
                                                                    glitch_tau_scan_file=glitch_ts_file,
                                                                    #gwb_log_amp_range=[-18,-15],
                                                                    rn_log_amp_range=[-18,-15],
                                                                    wavelet_log_amp_range=[-10.0,-5.8],
                                                                    per_psr_rn_log_amp_range=[-18,-11],
                                                                    #rn_params = [noise_params['gw_crn_log10_A'],noise_params['gw_crn_gamma']],
                                                                    prior_recovery=False,
                                                                    #gwb_amp_prior='log-uniform',
                                                                    rn_amp_prior='log-uniform',
                                                                    wavelet_amp_prior='uniform',
                                                                    per_psr_rn_amp_prior='log-uniform',
                                                                    #gwb_on_prior=0.975,
                                                                    max_n_glitch=3,
                                                                    #n_glitch_start='random',
                                                                    glitch_log_amp_range=[-10.0,-5.8],
                                                                    glitch_amp_prior='uniform',
                                                                    f0_max = f_max,
                                                                    f0_min = f_min,
                                                                    tau_max_in = tau_max,
                                                                    tau_min_in = tau_min,
                                                                    t0_max=t0_max,
                                                                    tref = tref,
                                                                    vary_white_noise=True,  
                                                                    include_rn=True, vary_rn=True,
                                                                    include_equad=True,
                                                                    include_ecorr=False,
                                                                    include_efac=True,
                                                                    wn_backend_selection=False,
                                                                    noisedict= noise_params,
                                                                    include_per_psr_rn=False,
                                                                    vary_per_psr_rn=False,
                                                                    #resume_from=savepath,
                                                                    #per_psr_rn_start_file=RN_start_file,
                                                                    n_fish_update = n_fish_update,
                                                                    savepath=savepath, save_every_n=100,
                                                                    n_fast_to_slow=1, thin = 100, ent_lnlike_test = True, ent_verbosity = True, QB_attributes = attributes)
                         
#np.savez(savefile, samples=samples, acc_fraction=acc_fraction, swap_record=swap_record, log_likelihood=log_likelihood)

NameError: name 'os' is not defined

In [ ]:
step_array[701]

In [ ]:
np.shape(step_array)

In [ ]:
n_chain = [n for n in range(len(acc_fraction[6]))]
print(n_chain)
for n in n_chain: 
    plt.figure(figsize = (7,5))
    plt.plot(ent_lnlikelihood[n], ent_lnlikelihood[n], ls='--', marker='.', color='xkcd:blue', label = 'Ent')
    plt.plot(ent_lnlikelihood[n], log_likelihood[n], ls='', marker='.', color='xkcd:green', label = 'FastB')
    plt.legend(loc='upper left')
    plt.xlabel("Enterprise LogL")
    plt.ylabel("QB LogL")
    plt.title('QuickBurst MCMC likelihood comparisons: chain {}'.format(n))
    plt.show()

In [ ]:
print(step_array[1000*10-9])
log_likelihood[0][100:] - ent_lnlikelihood[0][100:]

In [ ]:
for n in n_chain:
    plt.figure(figsize = (7,5))
    plt.plot(log_likelihood[n][:] - ent_lnlikelihood[n][:], ls='', marker='.', color='xkcd:blue')
    #plt.gca().axhline(0.1, ls='--', color='xkcd:green')
    plt.xlabel("index")
    plt.ylabel("delta log(likelihood)")
    #plt.ylim()
    plt.title('Difference in QB MCMC samples: chain {}'.format(n))
    plt.show()

## Run this to check steps for likelihoods that didn't match. 

In [ ]:
for n in n_chain:
    for i inn range(len(log_likelihood[0])):
        if log_likelihood[n][i] - ent_lnlikelihood[n][i] > 1e-9 or log_likelihood[n][i] - ent_lnlikelihood[n][i] < -1e-9:
            print(step_array[i])

### 9/16/23 Well, something is broken again, so we will have to return to troubleshooting techniques again. I've re-added in calculating enterprise likelihoods, and it's fairly stable for at least potentially 1e3 samples (though I only did 1e2 to preserve another timing run for the MCMC. 
### I THINK we can probably still use the timing data despite this, but depending on what broke, that data may change. 

In [ ]:
start_iter = 0
stop_iter = 10

# for i in range(start_iter, stop_iter-1):
#     print(i)
print(max(range(start_iter, stop_iter-1)))

# Running samples through enterprise PTAs to test for agreement
## Note: Can also do this by loading in chain and calling QuickBurst.get_pta() and simply returning the enterprise ptas. The parameter volume in get_pta() must match the run!

In [ ]:
import h5py
filename = "/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/chains/12p5yr_A4Cast_1g1w_noWN_speedups_enterprisetest.h5df"

with h5py.File(filename, 'r+') as f:
    n_chain = 2
    samples_array = f['samples_cold'][()]
    log_likelihood = f['log_likelihood'][()]
    #ent_lnlikelihood = f['ent_lnlikelihood'][()]
    acc_frac = f['acc_fraction'][()]
    param_names = f['par_names'][()]

In [ ]:
globvar = True


def print_globvar():
    print(globvar)
print_globvar()

In [ ]:
type(ent_ptas[1][1])

In [ ]:
max_n_glitch=1
max_n_wavelet=1
for i in range(max_n_wavelet+1):
    for j in range(max_n_glitch+1):
        with open("/home/reyna/BayesHopperBurst/QuickBurst/12p5_year_A4Cast/ent_ptas_test/{}w{}g_noWN_ent_ptas.json".format(i, j), "wb") as f:
                json.dump(ent_ptas[i][j], f)

In [ ]:
import time
n_chain = len(samples_array[0])
num_samples = len(samples_array[0, :, 0])
print('chain length: {}'.format(num_samples))
ent_lnlikelihoods = []
#loop through number of samples
starting_time = time.time()
for i in range(num_samples):
    if i%num_samples/10 or i == 0:
        print('Currently at {}% in {} seconds'.format(int(i/num_samples)*100, starting_time-time.time()))
    temp_lnlikelihoods = []
    for j in range(n_chain):
        n_wavelet = int(samples_array[j, i, 0])
        n_glitch = int(samples_array[j, i, 1])
        x0 = samples_array[j, i, 2:]
        print(n_wavelet, n_glitch, x0)
        print(ent_ptas[n_wavelet][n_glitch], '\n', ent_ptas[n_wavelet][n_glitch].params)
        temp_lnlikelihoods.append(ent_ptas[n_wavelet][n_glitch].get_lnlikelihood(x0))
    ent_lnlikelihoods.append(temp_lnlikelihoods)

## Take differences of likelihoods

In [ ]:
like_diff = []
for i in range(num_samples):
    temp_diffs = []
    for j in range(n_chain):
        temp_diffs.append(log_likelihood[j, i] - ent_lnlikelihoods[j, i])
    like_diff.append(temp_diffs)

In [ ]:
like_diff

# Testing for ShermanMorrison stuff below

In [ ]:
max_n_wavelet = 5
max_n_glitch = 5
n_wavelet_start = 1
n_glitch_start = 1
save_every_n = 100
num_params = max_n_wavelet*10+max_n_glitch*6
num_params += 2 #for keepeng a record of number of wavelets and glitches

vary_rn = True
vary_per_psr_rn = True


In [ ]:
noise_dict_sim = {} #new noise dict to conver the simulated noise file into a readable formate
for k,v in noise_params_sim.items():
    if '_equad' in k:
        noise_dict_sim.update({k.split('_equad')[0] + '_t2equad': v})
    else:
        noise_dict_sim.update({k : v})

In [ ]:
noise_dict_sim

In [ ]:
import QuickBurst_MCMC

PTA, ent_ptas, QB_FP, QB_FPI, glitch_indx, wavelet_indx, per_puls_wn_indx, per_puls_rn_indx, rn_indx, num_per_puls_wn_param_list = QuickBurst_MCMC.get_pta(psrs_sim, 
                                                                                                            vary_white_noise = False,
                                                                                                            max_n_wavelet=max_n_wavelet, 
                                                                                                            max_n_glitch=max_n_glitch,   
                                                                                                            include_equad = True, 
                                                                                                            include_ecorr = True, 
                                                                                                            wn_backend_selection=True, 
                                                                                                            noisedict=noise_dict_sim, 
                                                                                                            include_rn=True, 
                                                                                                            vary_rn=vary_rn, 
                                                                                                            include_per_psr_rn=True, 
                                                                                                            vary_per_psr_rn=vary_per_psr_rn, t0_max = t0_max, tref = tref) 

In [ ]:
num_per_psr_wn_params = 0
num_per_psr_rn_params = 0
num_noise_params = 0
#total # of wn params in pta
num_per_psr_wn_params += sum(num_per_puls_wn_param_list)

#CURN A and gamma
if vary_rn:
    num_noise_params += 2
if vary_per_psr_rn:
    #A and gamma per pulsar
    num_per_psr_rn_params += 2*len(psrs_sim)

#Total # of noise params
num_noise_params += num_per_psr_rn_params + num_per_psr_wn_params

#Total num params: per_puls_wn + per_puls_rn + CURN + wave/glitch
num_params += num_noise_params

In [ ]:
num_params

In [ ]:
samples = np.zeros((1, save_every_n+1, num_params))

print(np.shape(samples))
samples[0,0,0] = n_wavelet_start
samples[0,0,1] = n_glitch_start
samples[0,0,2:] =  np.hstack(p.sample() for p in PTA.params)

for param, idx in enumerate(PTA.param_names):
    if param in noise_dict_sim.keys():
        samples[j, 0, 2+idx] = noise_dict_sim[param]
        
for windx in range(max_n_wavelet):
    samples[0,0,2+int(wavelet_indx[windx,0])] = samples[0,0,2+int(wavelet_indx[0,0])]
    samples[0,0,2+int(wavelet_indx[windx,1])] = samples[0,0,2+int(wavelet_indx[0,1])]
    samples[0,0,2+int(wavelet_indx[windx,2])] = samples[0,0,2+int(wavelet_indx[0,2])]

In [ ]:
print(PTA.signals)

In [ ]:
n_wavelet = int(samples[0,0,0])
n_glitch = int(samples[0,0,1])
first_sample = np.copy(samples[0,0,2:])

#Generate first sample param dictionary
param_names = PTA.param_names
sample_dict = dict((k, v) for k, v in zip(param_names, first_sample))
# for i in range(len(first_sample)):
#     sample_dict[PTA.param_names[i]] = first_sample[i]
rn_check = False

if vary_per_psr_rn or vary_rn:
    rn_check = True

## Let's check the parameter types in QuickBurst using EcorrBasisModel for ecorr

In [ ]:
import time

In [ ]:
param_names = PTA.param_names[0:8]

In [ ]:
num_list = [1,2,3,4]

In [ ]:
test_dict = dict((k, v) for k, v in zip(param_names, num_list))

In [ ]:
starting_time = time.time()
param_names = PTA.param_names[0:40]
numberrr = 40
num_list = [x for x in range(numberrr)]
test_dict = dict((k, v) for k, v in zip(param_names, num_list))
print('time to make dict: ', time.time()-starting_time)

In [ ]:
starting_time = time.time()
param_names = PTA.param_names[0:40]
num_list = [x for x in range(numberrr)]
d0 = {}
for i in range(len(num_list)):
    d0[PTA.param_names[i]] = num_list[i]
print('time to make dict: ', time.time()-starting_time)

In [ ]:
Nvecs = list(PTA.get_ndiag(sample_dict))
TNTs = PTA.get_TNT(sample_dict)

#important matrixies
Nvecs = list(PTA.get_ndiag(sample_dict))
TNTs = PTA.get_TNT(sample_dict)
Ts = PTA.get_basis()

#pulsar information
toas = list([psr.toas - tref for psr in psrs_sim])
residuals = list([psr.residuals for psr in psrs_sim])
pos = np.zeros((len(psrs_sim),3))
for i in range(len(psrs_sim)):
    pos[i] = psrs_sim[i].pos

    logdet = 0
for (l,m) in PTA.get_rNr_logdet(sample_dict): #Only using this for logdet term because the rNr term removes the deterministic signal durring generation
    logdet += m

#terms used in cholesky component of the dot product (only needs to be updated per-pulsar)
invCholSigmaTN = []
phiinvs = PTA.get_phiinv(sample_dict, logdet=False, method='partition') #use enterprise to calculate phiinvs
for ii in range(len(psrs_sim)):
    TNT = TNTs[ii]
    T = Ts[ii]
    phiinv = phiinvs[ii]
    #print('self.Nvecs[ii]', Nvecs[ii][0:1000])
    Ndiag = 1/Nvecs[ii]
    Sigma = TNT + (np.diag(phiinv) if phiinv.ndim == 1 else phiinv)
    chol_Sigma,lower = sl.cho_factor(Sigma.T,lower=True,overwrite_a=True,check_finite=False)
    invchol_Sigma_T_loc = solve_triangular(chol_Sigma,T.T,lower_a=True,trans_a=False,overwrite_b=False)
    invCholSigmaTN.append(invchol_Sigma_T_loc)#*Ndiag)

In [ ]:
type(Nvecs[0])

In [ ]:
a = Nvecs[0]

In [ ]:
print(type(np.array(np.copy(a.data))))
print(type(a.copy))

In [ ]:
for idx, psr in enumerate(psrs_sim):
    print(psr.name, idx)

In [ ]:
print(np.shape(Nvecs[20]))